In [1]:
# Dataloader
import torch
import torchvision
import glob
from torch import nn
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.data import random_split

seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

transform = transforms.Compose([
    torchvision.transforms.Resize((384,384)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class DC_frame_loader(Dataset):
    def __init__(self, frame_root,ann_root,transform=None):
        # self.transforms = transforms.Compose([transforms.ToTensor(),transforms.Resize((128,64)),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.frames = sorted(glob.glob(os.path.join(frame_root,"*")))
        self.heatmaps = sorted(glob.glob(os.path.join(ann_root,"*")))
        self.size = len(self.heatmaps)
        self.transform=transform
        self.heatmap_transforms = transforms.Resize((384,384), Image.BILINEAR)
    def __len__(self):
        return self.size
    def __getitem__(self, index):
        # Load video frames
        frame = Image.open(self.frames[index])
        frame = self.transform(frame)
        # frame = frame.squeeze(dim=1)
        # Load ground truth heatmap
        heatmap = Image.open(self.heatmaps[index])
        heatmap = transforms.ToTensor()(heatmap)
        heatmap = heatmap.squeeze(dim=0)
        return frame, heatmap
frame_root = '/home/sohel_experiment/segmentation-data/Inputs_OriginalimagesTaskID_PNGs'
ann_root = '/home/sohel_experiment/segmentation-data/BinaryClassification'
dataset = DC_frame_loader(frame_root,ann_root,transform=transform)
train_size = int(0.7 * (round(len(dataset))))  # split in 0.7,0.3 for train and test.
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(seed))
train_len = len(train_dataset)
val_len = len(test_dataset)
print(train_len,val_len)
train_loader = DataLoader(train_dataset, batch_size=16,
                                shuffle=True,
                                num_workers=8,drop_last=True)
val_loader = DataLoader(test_dataset, batch_size=1,
                                shuffle=True,num_workers=0)


/home/sohel/anaconda3/envs/ML_challenge/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


137 59


/home/sohel/anaconda3/envs/ML_challenge/lib/python3.6/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [2]:
class diceloss(torch.nn.Module):
    def init(self):
        super(diceLoss, self).init()
    def forward(self,pred, target):
       smooth = 1.
       iflat = pred.contiguous().view(-1)
       tflat = target.contiguous().view(-1)
       intersection = (iflat * tflat).sum()
       A_sum = torch.sum(iflat * iflat)
       B_sum = torch.sum(tflat * tflat)
       return 1 - ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )

In [3]:
import segmentation_models_pytorch as deeplav3pl
import torch_optimizer

model = deeplav3pl.DeepLabV3Plus(encoder_name="resnet50",activation=None)
model = model.cuda()
critertion = diceloss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch_optimizer.RAdam(model.parameters(),lr= 0.01,betas=(0.9, 0.999),eps=1e-8,weight_decay=0)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50,200], gamma=0.1)

In [4]:
import matplotlib.pyplot as plt
def train_epoch(model,train_loader,optimizer,criterion,epoch,image_out_dir):
    model.train()
    losses=[]
    # train_acc=torchmetrics.Accuracy()
    j=0
    for i, data in enumerate(train_loader):
        inp, heatmap = data
        # inp = inp.permute(0,1,3,4,2) # b c h w t
        inp,heatmap = inp.cuda(),heatmap.cuda()
        out = model(inp)
        
        # out = out.squeeze(dim=4)
        out = out.squeeze(dim=1)
        out = nn.Sigmoid()(out)
        loss = criterion(out,heatmap)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if epoch % 5 == 0:
            Path(os.path.join(image_out_dir,str(epoch))).mkdir(parents=True, exist_ok=True)
            # for o,h_map in zip(out,heatmap):
            o = out[0]
            h_map = heatmap[0]
            plt.imshow(o.cpu().detach().numpy())
            plt.colorbar()
            plt.savefig(os.path.join(image_out_dir,str(epoch),str(i)+'_pred.png'))
            plt.close()
            plt.imshow(h_map.cpu().detach().numpy())
            plt.colorbar()
            plt.savefig(os.path.join(image_out_dir,str(epoch),str(i)+'_gt.png'))
            plt.close()
            j=j+1
        # train_acc(out.cpu(),gt_cls.cpu())    
    avg_losses = sum(losses)/train_len
    return avg_losses

In [5]:
def val_epoch(model,val_loader,criterion,epoch,image_out_dir):
    model.eval()
    losses=[]
    # val_acc=torchmetrics.Accuracy()
    j=0
    for i, data in enumerate(val_loader):
        inp, heatmap = data
        # inp = inp.permute(0,1,3,4,2)
        
        inp,heatmap = inp.cuda(),heatmap.cuda()
        with torch.no_grad():
            out = model(inp)
            # out = out.squeeze(dim=4)
            out = out.squeeze(dim=1)
            out = nn.Sigmoid()(out)
            loss = criterion(out,heatmap)        
        if epoch % 5 == 0:
            Path(os.path.join(image_out_dir,str(epoch))).mkdir(parents=True, exist_ok=True)
            # for o,h_map in zip(out,heatmap):
            o = out[0]
            h_map = heatmap[0]
            plt.imshow(o.cpu().detach().numpy())
            plt.colorbar()
            plt.savefig(os.path.join(image_out_dir,str(epoch),str(j)+'_pred.png'))
            plt.close()
            plt.imshow(h_map.cpu().detach().numpy())
            plt.colorbar()
            plt.savefig(os.path.join(image_out_dir,str(epoch),str(j)+'_gt.png'))
            plt.close()
                # j=j+1
        losses.append(loss.item())
        # val_acc(out.cpu(),gt_cls.cpu())
    avg_losses = sum(losses)/val_len
    # val_acc.reset()
    return avg_losses

In [6]:
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
Path(os.path.join("./results/tensorboard/train")).mkdir(parents=True, exist_ok=True)
Path(os.path.join("./results/tensorboard/val")).mkdir(parents=True, exist_ok=True)
train_img_out_dir = os.path.join("./results/image_outputs","train")
Path(train_img_out_dir).mkdir(parents=True, exist_ok=True)
val_img_out_dir = os.path.join("./results/image_outputs","val")
Path(val_img_out_dir).mkdir(parents=True, exist_ok=True)
train_writer = SummaryWriter(os.path.join("./results/tensorboard/train"))
val_writer = SummaryWriter(os.path.join("./results/tensorboard/val"))
for epoch in range(51):
    print(f'epoch={epoch+1}')
    train_losses = train_epoch(model,train_loader,optimizer,critertion,epoch,train_img_out_dir)
    val_losses = val_epoch(model,val_loader,critertion,epoch,val_img_out_dir)
    train_writer.add_scalar('loss',train_losses,epoch)
    val_writer.add_scalar('loss',val_losses,epoch)
    print(f'training losses={train_losses}, validation losses = {val_losses}')
    scheduler.step()

epoch=1
training losses=0.05781552869908131, validation losses = 0.9964432979034166
epoch=2
training losses=0.051276417544288355, validation losses = 0.9483432214138872
epoch=3
training losses=0.04117606591134176, validation losses = 0.9091445096468521
epoch=4
training losses=0.034261242751657525, validation losses = 0.8601473832534532
epoch=5
training losses=0.031394312851620414, validation losses = 0.6505672022447748
epoch=6
training losses=0.030173323885367736, validation losses = 0.6414766059083453
epoch=7
